# Transformers pour la classification de texte

L'objectif de ce TP est d'implémenter une version simplifiée d'un Transformer pour résoudre un problème de classification de texte.

Nous utiliserons comme exemple illustratif une base de données présente dans la librairie ```Keras``` consistant en des critiques de films postées sur le site IMDB, accompagnées d'une note qui a été binarisée pour révéler le caractèe positif, ou négatif, de la critique.

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Implémentation d'un bloc de base de Transformer


<center><img src="https://drive.google.com/uc?id=1leAVyTZJ2gZ26CFoauMtmu7T4Jo-Crc_" width=200> </center>
<caption><center> Figure 1: Schéma de l'architecture de BERT</center></caption>

La figure ci-dessus présente l'architecture de BERT. Le bloc de base d'un Transformer est composé d'un bloc de *Self-Attention*, d'une couche de ```Layer Normalization``` (similaire à la ```Batch Normalization```), d'une couche dense et enfin d'une nouvelle couche de ```Layer Normalization```.

Pour implémenter la *Self-Attention*, vous pouvez utiliser la fonction ```Multi-Head Attention``` (à vous de regarder quels en sont les paramètres dans la documentation).

**Rappel**: Une couche d'Attention *Multi-Head*  se présente sous la forme ci-dessous à gauche, avec le mécanisme d'attention détaillé à droite :


<center>

<img src="https://drive.google.com/uc?id=1UTozEHtsZ3xy61XJqn_Eug-7mn7bFp9m">
<img src="https://drive.google.com/uc?id=1aTttpp1OOasVVZAi3lWwosh68VnBjQnz">
</center>

D'après vous, combien de paramètres comporte une couche d'attention à 2 têtes, pour un *Embedding* de dimension 32 ?

In [6]:
class TransformerBlock(layers.Layer):
    # embed_dim désigne la dimension des embeddings maintenus à travers les différentes couches,
    # et num_heads le nombre de têtes de la couche d'attention.
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        # Définition des différentes couches qui composent le bloc
        # Couche d'attention
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        # Première couche de Layer Normalization
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        # Couche Dense (Feed-Forward)
        self.ffn = layers.Dense(units=embed_dim, activation='relu')
        # Deuxième couche de normalisation
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs, training):
        # Application des couches successives aux entrées
        # Couche d'attention
        x = self.att(inputs, inputs)
        # Ajout de la connexion résiduelle et normalisation
        x = self.layernorm1(inputs + x)

        # Couche Dense (Feed-Forward)
        ffn_output = self.ffn(x)
        # Ajout de la connexion résiduelle et normalisation
        x = self.layernorm2(x + ffn_output)

        return x

## Implémentation de la double couche d'Embedding

La séquence d'entrée est convertie en *Embedding* de dimension ```embed_dim```.
L'*Embedding* final est constitué de la somme de deux *Embedding*, le premier encodant un mot, et le second encodant la position du mot dans la séquence.

La couche d'*Embedding* de Keras (```layers.Embedding```) est une sorte de table associant à un indice en entrée un vecteur de dimension ```embed_dim```. Chaque coefficient de cette table est en fait un paramètre apprenable.

D'après vous combien de paramètres contiendrait une couche d'*Embedding* associant un vecteur de dimension 32 à chacun des 20000 mots les plus courants du vocabulaire extrait de la base de données que nous allons utiliser ?
Et combien pour l'*Embedding* qui associe un vecteur de dimension 32 à chaque position d'un séquence de longueur ```maxlen``` ?

In [3]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        # Définition des différentes couches qui composent le bloc Embedding
        # Embedding de mot
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)

        # Embedding de position
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        # Calcul de l'embedding à partir de l'entrée x
        # ATTENTION : UTILISER UNIQUEMENT DES FONCTIONS TF POUR CETTE PARTIE
        # Récupération de la longueur de la séquence
        maxlen = tf.shape(x)[-1]
        # Création d'un vecteur [0, 1, ..., maxlen] des positions associées aux
        # mots de la séquence
        positions = tf.range(start=0, limit=maxlen, delta=1)
        # Calcul des embeddings de position
        positions_emb = self.pos_emb(positions)
        # Calcul des embeddings de mot
        words_emb = self.token_emb(x)
        # Somme des embeddings de position et de mot pour obtenir l'embedding final
        x = words_emb + positions_emb
        return x

## Préparation de la base de données

In [4]:
# Taille du vocabulaire considéré (on ne conserve que les 20000 mots les plus courants)
vocab_size = 20000
# Taille maximale de la séquence considérée (on ne conserve que les 200 premiers mots de chaque commentaire)
maxlen = 200

# Chargement des données de la base IMDB
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)

print(len(x_train), "séquences d'apprentissage")
print(len(x_val), "séquences de validation")

# Padding des séquences : ajout de "0" pour compléter les séquences trop courtes
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)

25000 séquences d'apprentissage
25000 séquences de validation


## Création du modèle

Pour assembler le modèle final, il faut, partant d'une séquence de longueur ```maxlen```, calculer les Embedding puis les fournir en entrée d'une série de blocs Transformer. Pour ce TP, **commencez par ne mettre qu'un seul bloc Transformer**. Vous pourrez en ajouter plus tard si vous le souhaitez.

Pour construire la tête de projection du réseau, vous pouvez moyenner les activations en sortie du bloc Transformer par élément de la séquence grâce à un *Global Average Pooling* (1D !), à relier à une couche dense (par exemple comportant 20 neurones) et enfin à la couche de sortie du réseau.

In [10]:
embed_dim = 32  # Dimension de l'embedding pour chaque mot
num_heads = 2  # Nombre de têtes d'attention

# A COMPLETER
# Création du modèle
inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(20, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 200)]             0         
                                                                 
 token_and_position_embeddi  (None, 200, 32)           646400    
 ng_2 (TokenAndPositionEmbe                                      
 dding)                                                          
                                                                 
 transformer_block_2 (Trans  (None, 200, 32)           9600      
 formerBlock)                                                    
                                                                 
 global_average_pooling1d_1  (None, 32)                0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_4 (Dense)             (None, 20)                660 

Enfin vous pouvez lancer l'apprentissage, avec par exemple l'optimiseur Adam. Inutile de lancer de trop nombreuses *epochs*, le réseau sur-apprend très vite !

In [9]:
# A COMPLETER
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    x_train, y_train, batch_size=32, epochs=5, validation_data=(x_val, y_val)
)

Epoch 1/5
782/782 [==============================] - 86s 99ms/step - loss: 0.3666 - accuracy: 0.8303 - val_loss: 0.3701 - val_accuracy: 0.8345
Epoch 2/5
782/782 [==============================] - 25s 31ms/step - loss: 0.1913 - accuracy: 0.9266 - val_loss: 0.3771 - val_accuracy: 0.8535
Epoch 3/5
782/782 [==============================] - 17s 22ms/step - loss: 0.1279 - accuracy: 0.9545 - val_loss: 0.3575 - val_accuracy: 0.8634
Epoch 4/5
782/782 [==============================] - 15s 19ms/step - loss: 0.0821 - accuracy: 0.9728 - val_loss: 0.4913 - val_accuracy: 0.8514
Epoch 5/5
782/782 [==============================] - 13s 17ms/step - loss: 0.0563 - accuracy: 0.9816 - val_loss: 0.5575 - val_accuracy: 0.8494


**Questions subsidiaires**:



1.   Testez un LSTM bi-directionnel, comme nous l'avons vu dans le TP précédent, et comparez les résultats obtenus sur ce problème.

